In [18]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from collections import defaultdict
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

In [24]:
import warnings
warnings.filterwarnings("ignore")

## Данные

- app_id - Идентификатор заявки.
- amnt - Нормированная сумма транзакции
- currency - Идентификатор валюты транзакции
- operation_kind - Идентификатор типа транзакции
- card_type - Уникальный идентификатор типа карты
- operation_type - Идентификатор типа операции по пластиковой карте
- operation_type_group - Идентификатор группы карточных операций (дебетовая карта, кредитная карта)
- ecommerce_flag - Признак электронной коммерции
- payment_system - Идентификатор типа платежной системы
- income_flag - Признак списания/внесения денежных средств на карту
- mcc - Уникальный идентификатор типа торговой точки
- country - Идентификатор страны транзакции
- city - Идентификатор города транзакции
- mcc_category - Идентификатор категории магазина транзакции
- day_of_week - День недели, когда транзакция была совершена
- hour - Час, когда транзакция была совершена
- days_before - Количество дней до даты выдачи кредита
- weekofyear - Номер недели в году, когда транзакция была совершена
- hour_diff - Количество часов с момента прошлой транзакции для данного клиента
- transaction_number - Порядковый номер транзакции клиента

## Feature engineering

Сгенерированные признаки:
- Основые статистики для числовых признаков: min, max, std, mean, 25%, 50%, 75%
- Мода и количество уникальных значений для категориальных
- Счетчики для значений категориальных переменных
- Сумма amount отдельно для списаний и зачислений
- Все признаки выше для последних 20% транзакций
- Последние значения для всех признаков
- Отношение первого значения amount к последнему 

In [25]:
num_features = ['amnt', 'day_of_week', 'hour', 'days_before', 'weekofyear', 'hour_diff']
cat_features = ['currency', 'operation_kind', 'card_type', 'operation_type', 'operation_type_group', 
                'ecommerce_flag', 'payment_system', 'income_flag', 'mcc', 'country', 'city', 'mcc_category']

def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

def aggregate_features(data):
    n_transactions = data.groupby(['app_id'])['transaction_number'].count().to_frame(name='n_transactions')
    num_stat = data.groupby('app_id')[num_features].agg('describe')
    num_stat.columns = num_stat.columns.map(' '.join)
    cat_agg = data.groupby('app_id')[cat_features].agg(['nunique', get_mode])
    cat_agg.columns = cat_agg.columns.map(' '.join)
    cat_counts = []
    for feature in cat_features:
        cat_count = data.groupby(['app_id', feature])['transaction_number'].count().unstack(fill_value=0)
        cat_count.columns = [' '.join([feature, str(col)]) for col in cat_count.columns]
        cat_counts.append(cat_count)
    cat_counts = pd.concat(cat_counts, axis=1)
    amnt_type = data.groupby(['app_id', 'income_flag'])['amnt'].sum().unstack()
    amnt_type.columns = [' '.join(['amnt type', str(col)]) for col in amnt_type.columns]
    
    return pd.concat([n_transactions, num_stat, cat_agg, cat_counts, amnt_type], axis=1)     

In [26]:
def process_file(dir, filename):
    data = pd.read_parquet(os.path.join(dir, filename))
    main_features = aggregate_features(data)
    last_data = data.groupby(['app_id']).apply(lambda x: x.tail(round(0.2 *(len(x)))))
    last_features = aggregate_features(last_data.reset_index(drop=True))
    abs_in_last = list(set(main_features.columns) - set(last_features.columns))
    last_features[abs_in_last] = 0
    last_features.columns = 'last period ' + last_features.columns

    last_values = data.groupby(['app_id']).agg('last')
    last_values.columns = 'last ' + last_values.columns

    amnt = data.groupby(['app_id'])['amnt'].agg(['first', 'last'])
    amnt_first_last = (amnt['first'] / amnt['last']).to_frame(name='amnt_first_last')

    fin_data = pd.concat([main_features, last_features, last_values, amnt_first_last], axis=1)
    return fin_data

In [ ]:
dirname = '../default/train_transactions_contest'
all_data = []
for filename in os.listdir(dirname):
    all_data.append(process_file(dirname, filename))
train_data = pd.concat(all_data)

In [50]:
train_data = train_data.drop('app_id', axis=1)

In [51]:
target = pd.read_csv('train_target.csv')
train_data = train_data.merge(target, on='app_id')
train_data = train_data.fillna(0)

Возьмем 0 и 1 класс в равных пропорциях, чтобы избежать дисбаланса классов:

In [ ]:
train_0 = train_data[train_data['flag'] == 0]
train_1 = train_data[train_data['flag'] == 1]
sample0 = train_0.sample(n=26577, random_state=42).reset_index()
sample1 = train_1.sample(n=26577, random_state=42).reset_index()
train_sample = pd.concat([sample0, sample1])
train_sample = train_sample.sample(frac=1).reset_index(drop=True)
train_sample = train_sample.drop(['index'], axis=1)

## Feature selection

Удалим константные признаки:

In [30]:
train_sample = pd.read_csv('data/train_sample.csv')

In [31]:
constant_cols = train_sample.columns[train_sample.nunique() <= 1]
train_sample = train_sample.drop(constant_cols, axis=1)

И сильно скореллированные признаки:

In [32]:
y_data = train_sample['flag']
X_data = train_sample.drop('flag', axis=1)

In [33]:
correlations = defaultdict(list)
corr_mat = X_data.corr().values.tolist()
columns = X_data.columns
for ind_r, row in enumerate(corr_mat):
    for ind_c, col in enumerate(row):
        if ind_r != ind_c and corr_mat[ind_r][ind_c] > 0.95:
            correlations[columns[ind_r]].append(columns[ind_c])

In [34]:
features_to_drop = []
for feature, group in correlations.items():
    if feature not in features_to_drop:
        for feature2 in group:
            features_to_drop.append(feature2)

In [35]:
X_data = X_data.drop(features_to_drop, axis=1)

##### Select from model

In [36]:
X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=42)

In [37]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [38]:
f1_scores = []
all_models = []
for i, (train_ind, val_ind) in enumerate(skf.split(X.values, y)):
    X_train, X_val = X.iloc[train_ind, :], X.iloc[val_ind, :]
    y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]
    lgb_model = CatBoostClassifier(verbose=0, random_state=42)
    all_models.append(lgb_model)
    lgb_model.fit(X_train, y_train)
    val_preds = lgb_model.predict(X_val)
    f1 = f1_score(val_preds, y_val)
    f1_scores.append(f1)

In [39]:
importances = np.zeros(len(X.columns))
for model in all_models:
    importances += model.feature_importances_ / len(all_models)
importances = list(zip(X.columns, importances))
importances.sort(key=lambda x: x[1], reverse=True)
features_to_select = [feature for feature, imp in importances[:600]]

Топ признаков:

In [40]:
features_to_select[:20]

['hour_diff 50%',
 'hour std',
 'product',
 'hour_diff 75%',
 'hour_diff max',
 'amnt 25%',
 'mcc_category 9',
 'operation_kind 2',
 'operation_type 5',
 'days_before max',
 'last period hour_diff 50%',
 'amnt max',
 'mcc 36',
 'mcc nunique',
 'mcc_category 10',
 'mcc 33',
 'mcc_category nunique',
 'amnt 75%',
 'amnt mean',
 'hour max']

In [42]:
X_data = X_data[features_to_select]
train_sample = pd.concat([X_data, y_data], axis=1)
train_sample.to_csv('train_sample.csv', index=False)